# eMLP Inference Tutorial

In this tutorial, we will use a pretrained eMLP model to compute certain properties and perform MD simulations. We will start from one of the eQM7 models, which was trained by using data augmentation. The model is stored at the location `models/eQM7_aug1`. That model can be loaded as follows:

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np
import tensorflow as tf

In [ ]:
from emlp.longrange import LongrangeCoulomb, LongrangeEwald
from emlp.reference import ConstantFragmentsReference
from emlp.schnet import SchNet

In [ ]:
model = SchNet.from_restore_file('models/eQM7_aug1', longrange_compute = LongrangeCoulomb(), reference = ConstantFragmentsReference(float_type = tf.float64), float_type = 64, xla = False)

One can ignore the warnings about checkpointed values that are not bound. The arguments to load the models were explained in the notebook to train the eMLP. Here, it is important to stress that one can change the precision of the model, i.e. using *float32* or *float64*. If one wants to use the eMLP to perform geometry optimizations or use the Born-Oppenheimer mode of the eMLP, float64 is required. Otherwise, one can use float32 without any issues.

## Validating xyz files

As a first example, we will use the eMLP to compute energies and forces for a single configuration. First, we will load the first configuration from the validation set and filter the core centers:

In [ ]:
from emlp.help_functions import load_xyz, filter_cores
for data in load_xyz('data/validation.xyz'):
    all_numbers = data['Z']
    all_positions = data['pos']
    all_forces = data['force']
    energy = data['energy']
    efield = data['efield']
    break
    
centers = all_positions[np.where(all_numbers == 99)]
positions = all_positions[np.where(all_numbers != 99)]
numbers = all_numbers[np.where(all_numbers != 99)]
forces = all_forces[np.where(all_numbers != 99)]
centers = filter_cores(centers, positions, numbers)

Next, we will perform one static calculation (the electron pairs stay in their reference positions)

In [ ]:
output = model.compute_static(positions, numbers, centers, rvec = np.eye(3) * 100, efield = efield, list_of_properties = ['energy', 'forces'])
print(output)

By specifying the `list_of_properties` argument, one can choose what kind of properties that the eMLP will compute. For this specific configuration the energy and force MAE errors are:

In [ ]:
def MAE(x, y):
    return np.mean(np.abs(x - y))
print('MAE energy: ', MAE(energy, output['energy']), ' eV')
print('MAE forces: ', MAE(forces, output['forces']), ' eV/A')
print('MAE center forces: ', MAE(np.zeros(centers.shape), output['center_forces']), ' eV/A')

A dynamic simulation can be performed as well. There, the energy is minimized with respect to the electron pair positions:

In [ ]:
output = model.compute(positions, numbers, rvec = np.eye(3) * 100, efield = efield, init_centers = centers, list_of_properties = ['energy', 'forces'])
print(output)

Again, we can compute the errors and the electron pair displacements from the PBE0 reference positions:

In [ ]:
print('MAE energy: ', MAE(energy, output['energy']), ' eV')
print('MAE forces: ', MAE(forces, output['forces']), ' eV/A')
print('MAE center forces: ', MAE(np.zeros(centers.shape), output['center_forces']), ' eV/A')
print('MAE center displacement: ', MAE(centers, output['centers']), ' A')

Here, the error on the forces of the centers should be approximately zero, since we have optimized their positions. The eMLP equilibrium positions of the electron centers are stored in `output['centers']`.

## Molecular dynamics simulation

Molecular dynamics simulations with the eMLP can be done with [Yaff](https://github.com/molmod/yaff). Basically, the eMLP provides a Yaff wrapper around the `model.compute()` function (see above). Hence, it is possible to perform MD simulations with other codes when writing your own wrapper around that function. Here, in Yaff, an NVT simulation can be performed as follows:

In [ ]:
from yaff import *
from emlp.md import NVT
system = System(numbers, positions * angstrom)
NVT(system, model, steps = 100, centers = centers, screenprint = 10, nprint = 1, dt = 0.5, temp = 300,
    name = 'md_run1', efield = [0.0, 0.0, 0.0])

The code above should have performed an NVT simulation for `steps=100` steps, with a timestep of `dt` fs at a temperature of `temp`. It will write the positions and centers every `nprint` steps in an extended xyz file at the location `name`. Every `screenprint` steps, progress will be written to the screen.

## Computation of the Hessian matrix and stiffness constants

In this section, we will compute a full extended Hessian with the eMLP and use it to compute analytical stiffness constant of beta-glycine. In the `models` directory we have provided a pretrained model for beta-glycine. Since it is a periodic system, we will load our system as follows:

In [ ]:
model = SchNet.from_restore_file('models/beta', longrange_compute = LongrangeEwald(gcut=0.25, real_space_cancellation = True), reference = -3158.888879, float_type = 64)

First, we have to compute the equilibrium geometry and cell of beta-glycine with the eMLP. This should take approximately 120 optimization steps. Only after the optimization, the Hessian will be positive definite.

In [ ]:
for data in load_xyz('data/beta_glycine.xyz'):
    centers = data['pos'][np.where(data['Z'] == 99)]
    positions = data['pos'][np.where(data['Z'] != 99)]
    numbers = data['Z'][np.where(data['Z'] != 99)]
    rvec = data['Lattice'].reshape([3, 3])
N = positions.shape[0]
C = centers.shape[0]
print('Num atomic cores: ', N)
print('Num electron pairs: ', C)

In [ ]:
from emlp.md import Optimize
positions, centers, rvec = Optimize(model, positions, numbers, centers, rvec, log='opt.xyz', fullcell=True)

With the `fullcell` argument one can specify whether or not also optimize the energy with respect to the full cell matrix. Next, one should compute the full extended Hessian (this should take a few seconds on a GPU):

In [ ]:
full_hessian = model.compute_hessian(positions, numbers, centers, efield = [0, 0, 0], rvec = rvec, include_rvec = True, include_efield = True)
print(full_hessian.shape)

For a system with N atomic cores and C electron pairs, the Hessian matrix has a shape of 3N + 3C + 9 + 3 by 3N + 3C + 9 + 3. Since the 3N degrees of freedom of the atomic cores, the 3C degrees of freedom of the electron pairs, the 9 cell degrees of freedom and the 3 electric field degrees of freedom are included. Note that the latter 3 degrees of freedom are not necessary to compute the stiffness tensor (but are necessary to compute piezoelectric constants for instance).

In [ ]:
def to_voigt(C, symmetrize=True):
    ''' This functions symmetrizes the stiffness constant and converts it to voigt notation'''
    C = (C + np.transpose(C, (0, 1, 3, 2))) / 2.
    C = (C + np.transpose(C, (1, 0, 2, 3))) / 2.
    C = (C + np.transpose(C, (2, 3, 0, 1))) / 2.
    result = np.zeros([6, 6])
    key_map = {0: (0, 0), 1: (1, 1), 2: (2, 2), 3: (1, 2), 4: (0, 2), 5: (0, 1)}
    for i in range(6):
        for j in range(6):
            key_0, key_1 = key_map[i]
            key_2, key_3 = key_map[j]
            result[i, j] = C[key_0, key_1, key_2, key_3]
    return result

In [ ]:
def compute_stiffness_tensor(hessian, rvec, ridge=1e-10):
    ''' This function computes the stiffness constant from a 3N + 3C + 9 by 3N + 3C + 9 matrix'''
    haa = hessian[-9:,-9:]
    haa = (haa + haa.T) / 2.
    haf = (hessian[-9:,:-9] + hessian[:-9, -9:].T) / 2.
    hff = hessian[:-9,:-9]
    hff = (hff + hff.T) / 2.

    evals, evecs = np.linalg.eigh(hff)
    hafevecs = np.dot(haf, evecs)
    evalsinv = evals / (evals**2+ridge**2)
    effective_hessian = haa - np.dot(hafevecs*evalsinv, hafevecs.T)
    np.testing.assert_allclose(effective_hessian, effective_hessian.T)
    
    volume = np.linalg.det(rvec)
    
    C = np.einsum('mk,gi,gjml->ijkl', rvec, rvec, np.reshape(effective_hessian, [3, 3, 3, 3]), optimize=True) / volume
    C *= electronvolt / angstrom**3 / (1e+09 * pascal)
    return C # [3, 3, 3, 3]

In [ ]:
T = N + C
stiffness_tensor = compute_stiffness_tensor(full_hessian[:T*3 + 9, :T*3+9], rvec)
stiffness_tensor_voigt = to_voigt(stiffness_tensor)
print(np.diag(stiffness_tensor_voigt))

The six numbers above are the diagonal components of the stiffness tensor of beta_glycine in GPa. More information about how to compute the tensor analytically can be found in the eMLP paper.